In [6]:
import numpy as np 
import pandas as pd 
import yfinance as yf 
import datetime as dt 
from _measurements import * 
from _optimization import *
from _interval_optimization import *
import seaborn as sns 
import matplotlib.pyplot as plt

sns.set_theme()

In [7]:
%reload_ext autoreload
%autoreload 2

In [8]:
# List of symbols
symbols = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'JNJ', 'WMT', 'KO', 'BAC', 'ADBE', 'CSCO', 'MCD', 'NFLX', 'NKE', 'PUM.DE',
    'PM', 'IBM', 'GE'
]

# Download historical data for the symbols
data = yf.download(symbols, start='2007-01-01', end='2023-08-07')["Close"]

# Print the first few rows of the data
data.dropna(inplace=True)
returns = data.pct_change().dropna()
returns

[*********************100%%**********************]  18 of 18 completed


18 Failed downloads:
['WMT', 'BAC', 'IBM', 'GOOGL', 'JNJ', 'AMZN', 'PM', 'MCD', 'NVDA', 'PUM.DE', 'MSFT', 'AAPL', 'KO', 'GE', 'ADBE', 'CSCO', 'NFLX', 'NKE']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2007-01-01 -> 2023-08-07)')


,AAPL,ADBE,AMZN,BAC,CSCO,GE,GOOGL,IBM,JNJ,KO,MCD,MSFT,NFLX,NKE,NVDA,PM,PUM.DE,WMT
Date,,,,,,,,,,,,,,,,,,


In [ ]:
methods_mu = ["historical", "EWMA", "ledoit", "oracle", "shrunk", "mincovdet"]
methods_cov = ["historical", "EWMA", "ledoit", "oracle", "shrunk", "mincovdet"]
objectives = ["min_risk", "max_sharpe", "max_return", "max_utility"]

final_results = []

for i in range(len(methods_mu)):
    mu = methods_mu[i]
    cov = methods_cov[i]
    for objective in objectives:
        params = {"train_low": 30, 
                "train_high": 33, 
                "test_low": 30, 
                "test_high": 33, 
                "step": 1,
                "method_mu": mu, 
                "method_cov": cov, 
                "objective": objective}
        
        int_opt = IntervalOptimization(returns, params)
        results = int_opt()
        final_results.append(results)

In [ ]:
np.array(final_results).shape